# Basic Usage of QuantLib analytics library
More details at:
https://quantlib-python-docs.readthedocs.io/en/latest/

In [1]:
import QuantLib as ql
import numpy as np
import pandas as pd
import datetime as dt

## 1. Objects and Handles
### Define a quote object and inspect the value.

In [2]:
quote = ql.SimpleQuote(.01)
quote.value()

0.01

### Use setValue() to reset values. Values will be propagated downstream in the dependency tree (using handles/smart pointers).

In [3]:
quote.setValue(.02)
quote.value()

0.02

### Define quoteHandle as a handle/smart pointer to the quote object.

In [4]:
quoteHandle = ql.QuoteHandle(quote)
quoteHandle.value()

0.02

### When the quote object is changed, the quoteHandle changes value as well.

In [5]:
quote.setValue(0.05)
quoteHandle.value()

0.05

## 2. Cashflow Schedules
### Construct a semi-annual fixed rate cashflow schedule object.

In [6]:
issue_date = ql.Date(17, 12, 2022)
maturity_date = ql.Date(17, 12, 2027)
coupon_freq = ql.Semiannual
coupon_term = ql.Period(coupon_freq)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
day_count_conv = ql.Unadjusted
date_generation = ql.DateGeneration.Backward
month_end = True
schedule = ql.Schedule(issue_date,
                       maturity_date,
                       coupon_term,
                       calendar,
                       day_count_conv,
                       day_count_conv,
                       date_generation,
                       month_end)

In [7]:
coupon_term

Period("6M")

### Inspect the cashflow schedule
- Use list() to get a list of all the dates in Schedule, and len() to get number of dates
- Use [] for random access
- use startDate(), endDate()

In [8]:
print("All dates: ", list(schedule))
print("Length: ", len(schedule))
print("The 3rd coupon date: ", schedule[3])  # random access
print("Start Date: ", schedule.startDate())
print("End Date: ", schedule.endDate())

All dates:  [Date(17,12,2022), Date(17,6,2023), Date(17,12,2023), Date(17,6,2024), Date(17,12,2024), Date(17,6,2025), Date(17,12,2025), Date(17,6,2026), Date(17,12,2026), Date(17,6,2027), Date(17,12,2027)]
Length:  11
The 3rd coupon date:  June 17th, 2024
Start Date:  December 17th, 2022
End Date:  December 17th, 2027


## 3. Discount Curve / Yield Curve Term Structure
### Constructing a Flat Yield Curve

In [9]:
calc_date = ql.Date(14, 4, 2023)
ql.Settings.instance().evaluationDate = calc_date

# using 5% flat interest rate for testing
flat_rate = ql.SimpleQuote(0.05)
rate_handle = ql.QuoteHandle(flat_rate)
day_count = ql.Actual360()
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
continuous_comp = ql.Continuous # continously compounded rate of 5%
flat_yield_curve = ql.FlatForward(calc_date, rate_handle, day_count, continuous_comp)
flat_yield_curve_handle = ql.YieldTermStructureHandle(flat_yield_curve)

In [14]:
flat_yield_curve

<QuantLib.QuantLib.FlatForward; proxy of <Swig Object of type 'ext::shared_ptr< FlatForward > *' at 0x7fdfc9d4bf00> >

### Inspecting the discount curve

In [10]:
ref_date = flat_yield_curve.referenceDate()
test_date = ql.Date(31, 12, 2023)

# calc year fraction between ref_date and test_date
yearFrac = flat_yield_curve.dayCounter().yearFraction(ref_date, test_date)

print("Reference Date =", ref_date)
print("Test Date =", test_date)
print("Year Fraction between Reference Date and Test Date : ", yearFrac)
print("Discount Factor for Test Date", test_date, ": ", flat_yield_curve.discount(test_date))
print("Manual DF calculation for Test Date", test_date, ": ", np.exp(-flat_rate.value() * yearFrac))
print("Difference in Discount Factor:", flat_yield_curve.discount(test_date) - np.exp(-flat_rate.value() * yearFrac))



Reference Date = April 14th, 2023
Test Date = December 31st, 2023
Year Fraction between Reference Date and Test Date :  0.725
Discount Factor for Test Date December 31st, 2023 :  0.9643991635522495
Manual DF calculation for Test Date December 31st, 2023 :  0.9643991635522495
Difference in Discount Factor: 0.0


## 4. Fixed Rate Bonds
### Constructing a bond object

In [11]:
coupon_rate = 0.04
day_count = ql.Thirty360(ql.Thirty360.USA)
coupons = [coupon_rate]
payment_convention = ql.Unadjusted

# Construct the FixedRateBond
settlement_days = 1
face_value = 100
fixed_rate_bond = ql.FixedRateBond(
    settlement_days,
    face_value,
    schedule,
    coupons,
    day_count,
    payment_convention)

### Investigate Bond Cashflows

In [12]:
for cf in fixed_rate_bond.cashflows():

    print(cf.amount())

2.0000000000000018
2.0000000000000018
2.0000000000000018
2.0000000000000018
2.0000000000000018
2.0000000000000018
2.0000000000000018
2.0000000000000018
2.0000000000000018
2.0000000000000018
100.0


In [13]:
x = [(cf.date(), cf.amount()) for cf in fixed_rate_bond.cashflows()]
cf_date, cf_amount = zip(*x)
cf_frame = pd.DataFrame(data={'CashFlowDate': cf_date, 'CashFlowAmount': cf_amount})
print(cf_frame)

           CashFlowDate  CashFlowAmount
0       June 17th, 2023             2.0
1   December 17th, 2023             2.0
2       June 17th, 2024             2.0
3   December 17th, 2024             2.0
4       June 17th, 2025             2.0
5   December 17th, 2025             2.0
6       June 17th, 2026             2.0
7   December 17th, 2026             2.0
8       June 17th, 2027             2.0
9   December 17th, 2027             2.0
10  December 17th, 2027           100.0


## 5. Present Value Calculation (no credit risk)
### Direct function call using risk-free bond pricing engine

In [28]:
bond_engine = ql.DiscountingBondEngine(flat_yield_curve_handle)
fixed_rate_bond.setPricingEngine(bond_engine)
pv_engine = fixed_rate_bond.NPV()
pv_engine

96.61211292221057

In [29]:
cf_frame

,CashFlowDate,CashFlowAmount
0,"June 17th, 2023",2.0
1,"December 17th, 2023",2.0
2,"June 17th, 2024",2.0
3,"December 17th, 2024",2.0
4,"June 17th, 2025",2.0
5,"December 17th, 2025",2.0
6,"June 17th, 2026",2.0
7,"December 17th, 2026",2.0
8,"June 17th, 2027",2.0
9,"December 17th, 2027",2.0


### Manual Calculation to verify PV

In [30]:
calc_frame = cf_frame
discount_yearfrac = np.zeros((len(calc_frame,)))
discount_factor = np.zeros((len(calc_frame,)))

print(calc_frame)

i = 0
for cf_date in calc_frame['CashFlowDate']:
    discount_yearfrac[i] = flat_yield_curve.dayCounter().yearFraction(flat_yield_curve.referenceDate(), cf_date)
    discount_factor[i] = flat_yield_curve.discount(cf_date)
    i += 1

print(calc_frame)

calc_frame['YearFrac'] = discount_yearfrac
calc_frame['DiscountFactor'] = discount_factor
calc_frame['NPV'] = calc_frame['CashFlowAmount'] * calc_frame['DiscountFactor']

calc_frame

           CashFlowDate  CashFlowAmount
0       June 17th, 2023             2.0
1   December 17th, 2023             2.0
2       June 17th, 2024             2.0
3   December 17th, 2024             2.0
4       June 17th, 2025             2.0
5   December 17th, 2025             2.0
6       June 17th, 2026             2.0
7   December 17th, 2026             2.0
8       June 17th, 2027             2.0
9   December 17th, 2027             2.0
10  December 17th, 2027           100.0
           CashFlowDate  CashFlowAmount
0       June 17th, 2023             2.0
1   December 17th, 2023             2.0
2       June 17th, 2024             2.0
3   December 17th, 2024             2.0
4       June 17th, 2025             2.0
5   December 17th, 2025             2.0
6       June 17th, 2026             2.0
7   December 17th, 2026             2.0
8       June 17th, 2027             2.0
9   December 17th, 2027             2.0
10  December 17th, 2027           100.0


,CashFlowDate,CashFlowAmount,YearFrac,DiscountFactor,NPV
0,"June 17th, 2023",2.0,0.177778,0.991151,1.982301
1,"December 17th, 2023",2.0,0.686111,0.966276,1.932552
2,"June 17th, 2024",2.0,1.194444,0.942026,1.884052
3,"December 17th, 2024",2.0,1.702778,0.918385,1.836769
4,"June 17th, 2025",2.0,2.208333,0.895461,1.790922
5,"December 17th, 2025",2.0,2.716667,0.872988,1.745976
6,"June 17th, 2026",2.0,3.222222,0.851197,1.702395
7,"December 17th, 2026",2.0,3.730556,0.829836,1.659671
8,"June 17th, 2027",2.0,4.236111,0.809122,1.618244
9,"December 17th, 2027",2.0,4.744444,0.788816,1.577632


In [31]:
pv_manual = calc_frame['NPV'].sum()
print('NPV_Engine = ', pv_engine)
print('NPV_Manual = ', pv_manual)
print('NPV diff = ', pv_manual - pv_engine)

NPV_Engine =  96.61211292221057
NPV_Manual =  96.61211292221057
NPV diff =  0.0


### Bond Clean vs Dirty Prices

In [32]:
print('Bond Notional = ', fixed_rate_bond.notional())
print('Settle Date = ', fixed_rate_bond.settlementDate())
print('Discount Factor to Settle Date = ', flat_yield_curve_handle.discount(fixed_rate_bond.settlementDate()))
print('Bond NPV (Calc Date) = ', fixed_rate_bond.NPV())
print('Bond Adjusted NPV (Settle Date) = ', fixed_rate_bond.NPV() / flat_yield_curve_handle.discount(fixed_rate_bond.settlementDate()))
print('Bond Dirty Price = ', fixed_rate_bond.dirtyPrice())
print('Bond Clean Price = ', fixed_rate_bond.cleanPrice())
print('Bond Accrued = ', fixed_rate_bond.accruedAmount())


Bond Notional =  100.0
Settle Date =  April 17th, 2023
Discount Factor to Settle Date =  0.9995834201268338
Bond NPV (Calc Date) =  96.61211292221057
Bond Adjusted NPV (Settle Date) =  96.65237635689454
Bond Dirty Price =  96.65237635689454
Bond Clean Price =  95.31904302356119
Bond Accrued =  1.333333333333342


## 6. Market Data Scenarios
### Parallel Shift in Interest Rates Curve (can be used to compute IR01/DV01 and convexity)

In [ ]:
# start with interest_rate_bump = 0
interest_rate_bump = ql.SimpleQuote(0.0)
flat_yield_curve_bumped = ql.ZeroSpreadedTermStructure(flat_yield_curve_handle, ql.QuoteHandle(interest_rate_bump))

bond_engine = ql.DiscountingBondEngine(ql.YieldTermStructureHandle(flat_yield_curve_bumped))
fixed_rate_bond.setPricingEngine(bond_engine)

# Original price (zero interest rate bump)
print("PV for spread = 0: ", fixed_rate_bond.NPV())

# New price for +10bps interest rate  bump
interest_rate_bump.setValue(0.0010)
print("PV for spread = +10bps: ", fixed_rate_bond.NPV())

# New price for -10bps interest rate  bump
interest_rate_bump.setValue(-0.0010)
print("PV for spread = -10bps: ", fixed_rate_bond.NPV())

interest_rate_bump.setValue(0)

PV for spread = 0:  96.59009015976352
PV for spread = +10bps:  96.1750271857446
PV for spread = -10bps:  97.00706466080435


## 7. Yield to Price conversions


In [ ]:
# Use original interest rate yield of 5%
# flat_rate.setValue(0.05)
print('Bond PV for', flat_rate.value()*100, 'pct yield:', fixed_rate_bond.NPV())


# Change interest rate yield to 6% and recompute bond PV
flat_rate.setValue(0.06)
print('Bond PV for', flat_rate.value()*100, 'pct yield:', fixed_rate_bond.NPV())

# Set interest rate yield back to 5%
flat_rate.setValue(0.05)

Bond PV for 5.0 pct yield: 96.59009015976352
Bond PV for 6.0 pct yield: 92.5240234751331


## 8. Price to Yield conversions


In [ ]:
# bond_market_price = fixed_rate_bond.cleanPrice()
bond_market_price = 95.00   # Clean market price

compounding = ql.Compounded
# compounding = ql.Continuous

settle_date = fixed_rate_bond.settlementDate(calc_date)
day_counter = fixed_rate_bond.dayCounter()

print('day_counter =', day_counter)
print('coupon_freq =', coupon_freq)
print('calc_date =', calc_date)
print('settle_date =', settle_date)


implied_yield = fixed_rate_bond.bondYield(bond_market_price, day_counter, compounding, coupon_freq, settle_date) * 100
print('implied_yield =', implied_yield)

day_counter = 30/360 (US) day counter
coupon_freq = 2
calc_date = April 14th, 2023
settle_date = April 17th, 2023
implied_yield = 5.219710350036621


## 9. Duration, Convexity and Z-Spread (flat yield model)

In [ ]:
# flat_int_rate
flat_int_rate = ql.InterestRate(flat_rate.value(), day_count, compounding, coupon_freq)

# Calc Duration and Convexity
bond_duration = ql.BondFunctions.duration(fixed_rate_bond, flat_int_rate)
bond_convexity = ql.BondFunctions.convexity(fixed_rate_bond, flat_int_rate)

# Calc z-spread for a given market price
bond_market_price = 95.00   # Clean market price
bond_zspread = ql.BondFunctions.zSpread(fixed_rate_bond, bond_market_price, flat_yield_curve, day_count, compounding, coupon_freq, settle_date)

# Print results
print('Bond Duration =', bond_duration)
print('Bond Convexity =', bond_convexity)
print('Bond Z-Spread bps =', round(bond_zspread * 10000, 2))


Bond Duration = 4.133759633020641
Bond Convexity = 20.24645403219985
Bond Z-Spread bps = 7.9


In [ ]:
def calc_clean_price_with_zspread(fixed_rate_bond, yield_curve_handle, zspread):
    zspread_quote = ql.SimpleQuote(zspread)
    zspread_quote_handle = ql.QuoteHandle(zspread_quote)
    yield_curve_bumped = ql.ZeroSpreadedTermStructure(yield_curve_handle, zspread_quote_handle, ql.Compounded, ql.Semiannual)
    yield_curve_bumped_handle = ql.YieldTermStructureHandle(yield_curve_bumped)
    
    # Set Valuation engine
    bond_engine = ql.DiscountingBondEngine(yield_curve_bumped_handle)
    fixed_rate_bond.setPricingEngine(bond_engine)
    bond_clean_price = fixed_rate_bond.cleanPrice()
    return bond_clean_price


In [ ]:
# Compare the original and the z-spread computed clean prices
bond_zspread_price = calc_clean_price_with_zspread(fixed_rate_bond, flat_yield_curve_handle, bond_zspread)

print('Bond Z-Spread bps =', round(bond_zspread * 10000, 2))
print('bond_market_price =', bond_market_price)
print('bond_zspread_price =', bond_zspread_price)
print('bond price diff =', bond_zspread_price - bond_market_price)

Bond Z-Spread bps = 7.9
bond_market_price = 95.0
bond_zspread_price = 95.00000000000034
bond price diff = 3.410605131648481e-13


## 10. Fixed rate bond helpers (needed for yield curve calibration)


In [ ]:
# clean price quote
tsy_clean_price_quote = 95.0
tsy_clean_price_handle = ql.QuoteHandle(ql.SimpleQuote(tsy_clean_price_quote))

# Create BondHelper object
bond_helper = ql.BondHelper(
    tsy_clean_price_handle,
    fixed_rate_bond)

print(bond_helper)

<QuantLib.QuantLib.BondHelper; proxy of <Swig Object of type 'ext::shared_ptr< BondHelper > *' at 0x000002078AD3C990> >
